In [ ]:
# Imports
import xpress as xp
import networkx as nx

# NetworkX tutorial available here: 
# https://networkx.org/documentation/stable/tutorial.html

In [2]:
# Example input data

pairs = ["P1", "P2", "P3", "P4"] 
altruistic_donors = ["NDD1"]
nodes = pairs + altruistic_donors
edges = {("NDD1", "P1"): 2,
         ("P1", "P2"): 10, 
         ("P2", "P3"): 10,
         ("P3", "P4"): 10,
         ("P4", "P1"): 10
}

In [3]:
# Create Xpress Model
# Initialize the model
prob = xp.problem()

# Define decision variables for each edge
y = {e: xp.var(vartype=xp.binary, name=f"y_{e[0]}_{e[1]}") for e in edges}
prob.addVariable(list(y.values()))

# Objective: Maximize total benefit
prob.setObjective(xp.Sum(y[e] * w for e, w in edges.items()), sense=xp.maximize)

# Constraints
for v in pairs:
    prob.addConstraint(xp.Sum(y[e] for e in edges if e[0] == v) <= xp.Sum(y[e] for e in edges if e[1] == v))
    prob.addConstraint(xp.Sum(y[e] for e in edges if e[1] == v) <= 1)

for a in altruistic_donors:
    prob.addConstraint(xp.Sum(y[e] for e in edges if e[0] == a) <= 1)

# Solve the model
prob.solve()

opt_sol = prob.getSolution()

# Print the output
print("")
print("")
print("")

print("Optimal Matches:")
for (u, v), var in y.items():
    if prob.getSolution(var) > 0.5:
        print(f"{u} donates to {v} with benefit {edges[(u,v)]}")

print(f"Total Benefit: {prob.getObjVal()}")

FICO Xpress v9.4.2, Hyper, solve started 9:57:52, Feb 17, 2025
Heap usage: 389KB (peak 389KB, 85KB system)
Maximizing MILP noname using up to 16 threads and up to 15GB memory, with these control settings:
OUTPUTLOG = 1
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = -1
Original problem has:
         9 rows            5 cols           15 elements         5 entities
Presolved problem has:
         0 rows            0 cols            0 elements         0 entities
Presolve finished in 0 seconds
Heap usage: 393KB (peak 403KB, 85KB system)
Will try to keep branch and bound tree memory usage below 8.9GB
Starting concurrent solve with dual (1 thread)

 Concurrent-Solve,   0s
            Dual        
    objective   dual inf
 D  40.000000   .0000000
                        
------- optimal --------
Concurrent statistics:
           Dual: 0 simplex iterations, 0.00s
Optimal solution found
 
   Its         Obj Value      S   Ninf  Nneg   Sum Dual Inf  Time
     0         40.000000      

C:\Users\ajmac\AppData\Local\Temp\ipykernel_39056\3367280310.py:3: LicenseWarning: Using the license file found in your Xpress installation. If you want to use this license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('C:/Program Files/xpressmp/bin/xpauth.xpr')

  prob = xp.problem()


In [52]:
# Check for long cycles
# Currently using this cell to understand how to detect *any* cycles.

# Create a directed graph with vertices 0, 1, 2, 3, 4, 5 and edges from i to i+1
DG = nx.DiGraph()
DG.add_edges_from([(i,i+1) for i in range(5)])

# Check the in-degree of the vertices:
print(f'Vertex in-degrees: {[DG.in_degree[i] for i in range(6)]}')

# Check the out-degree of the vertices:
print(f'Vertex out-degrees: {[DG.out_degree[i] for i in range(6)]}')

# Check the degree of the vertices:
print(f'Vertex degrees: {[DG.degree[i] for i in range(6)]}')
# ^ This gives the SUM of the in- and out-degree (in a directed graph).

Vertex in-degrees: [0, 1, 1, 1, 1, 1]
Vertex out-degrees: [1, 1, 1, 1, 1, 0]
Vertex degrees: [1, 2, 2, 2, 2, 1]


In [ ]:
# We can access the neighbors of vertices (out-neighbors):
print(f'The successors of vertex 1 are: {DG.adj[1]}.')

# In a _directed_ graph, these are equivalent to using successors:
print(f'The successors of vertex 1 are {list(DG.successors(1))}.\
      These require list() to be used to view like this.')

# The same object can be accessed with this callable function:
print(DG.neighbors(1))
# I'm not sure why this returns the object rather than giving the dictionary.

# Or simply by looking at the relevant 'element' of DG:
print(f'The successors of vertex 1: {DG[1]}')

The successors of vertex 1 are: {2: {}}.
The successors of vertex 1 are [2].      These require list() to be used to view like this.
The successors of vertex 1: {2: {}}


In [58]:
# To get the in-neighbors, we can use predecessors:
print(f'The predecessors of vertex 1 are {list(DG.predecessors(1))}.')

The predecessors of vertex 1 are [0].


In [51]:
# Fast examination of all (node, adjacency) pairs is achieved using 
# G.adjacency(), or G.adj.items().
print(list(DG.adjacency())) # Get all of the edges

print(DG.adj.items())

[(0, {1: {}}), (1, {2: {}}), (2, {3: {}}), (3, {4: {}}), (4, {5: {}}), (5, {})]
ItemsView(AdjacencyView({0: {1: {}}, 1: {2: {}}, 2: {3: {}}, 3: {4: {}}, 4: {5: {}}, 5: {}}))


In [65]:
try:
    nx.find_cycle(DG, orientation='original')
except:
    print("NO CYCLES")

NO CYCLES


https://networkx.org/documentation/stable/auto_examples/algorithms/plot_cycle_detection.html#sphx-glr-auto-examples-algorithms-plot-cycle-detection-py

In [67]:
list(nx.simple_cycles(DG))

[]

In [ ]:
# Create a DiGraph with cycles
DG2 = nx.DiGraph()
DG2.add_edges_from([(i,i+1) for i in range(5)])
DG2.add_edges_from([(3,1),(4,1)])

In [ ]:
# Find the cycles using nx.simple_cycles()
# DO NOT USE nx.recursive_simple_cycles(), which is high RAM.
list(nx.simple_cycles(DG2))

[[1, 2, 3, 4], [1, 2, 3]]

### NetworkX gotchas

Note that adding a node to G.nodes does not add it to the graph, use 
G.add_node() to add new nodes. Similarly for edges.

If you weight edges, the degree of a node is the sum of the weights of the
edges adjacent at that node, rather than just the number of edges adjacent.

In [ ]:
# Add constraint to remove long cycles